# CoreMS Tutorial
`CoreMS` framework for mass spectrum

In [11]:
from pathlib import Path
from corems.mass_spectrum.input.massList import ReadMassList
from corems.transient.input.brukerSolarix import ReadBrukerSolarix
# if using outside the docker container append corems path to sys.path and change the file location: 
file_location = Path.cwd() / "ESI_NEG_SRFA.d"

reader_obj =ReadBrukerSolarix(file_location)

bruker_transent = reader_obj.get_transient()

#reader_obj = ReadMassList(file_location, isCentroid=True)
#reader_obj = ReadMassList(file_location)
#reader_obj = ReadMassList(file_location)
#reader_obj = ReadHDF_BoosterMassSpectrum(file_location, polarity)
reader_obj.file_location

/Users/eber373/Desenvolvimento/corems/doc/notebooks


FileNotFoundError: File does not exist: /Users/eber373/Desenvolvimento/corems/doc/notebooks/ESI_NEG_SRFA.d

### 1. Create the reader object

### 2. Create the mass spectrum object


In [ ]:
mass_spectrum = bruker_transient.get_mass_spectrum(plot_result=False, auto_process=True)
mass_spectrum.mz_exp, mass_spectrum.abundance


### 3. Access mass spectrum peak objects

In [ ]:
kb = {'C': 1, 'H': 2}

for mspeak in mass_spectrum.sort_by_abundance():
    
    print(mspeak.mz_exp, mspeak.abundance, mspeak.resolving_power, 
          mspeak.signal_to_noise, mspeak.kmd, mspeak.is_assigned)
    
    mspeak.change_kendrick_base(kb)



### 4. How to change settings

In [ ]:
from corems.encapsulation.settings.io import settings_parsers

# Make chages on SettingsCoreMS.json before generate the mass spectrum file and load it:
# Looks for "SettingsCoreMS.json" at directory from where the code is being executed
# If SettingsCoreMS.json is not in the same directory pass parameters_path= "path location of SettingsCoreMS.json"

settings_parsers.load_search_setting_json()    

# Option 2. After mass spectrum object is generate you can manually change the settings inside the object:

mass_spectrum.molecular_search_settings.error_method = 'None'
mass_spectrum.molecular_search_settings.min_ppm_error  = -1
mass_spectrum.molecular_search_settings.max_ppm_error = 1
mass_spectrum.molecular_search_settings.mz_error_range = 1

mass_spectrum.molecular_search_settings.usedAtoms['C'] = (1,90)
mass_spectrum.molecular_search_settings.usedAtoms['H'] = (4,200)
mass_spectrum.molecular_search_settings.usedAtoms['O'] = (0,20)
mass_spectrum.molecular_search_settings.usedAtoms['N'] = (0,0)
mass_spectrum.molecular_search_settings.usedAtoms['S'] = (0,0)

# Option 3. make changes on SettingsCoreMS.json and call the function:

settings_parsers.load_setting_ms_obj(mass_spectrum)

### 5. Mass spectrum calibration

In [ ]:
from corems.molecular_id.search.findOxygenPeaks import FindOxygenPeaks
from corems.mass_spectrum.calc.calibrationCalc import FreqDomain_Calibration




find_formula_thread = FindOxygenPeaks(mass_spectrum)
find_formula_thread.start()
find_formula_thread.join()

mspeaks_results = find_formula_thread.get_list_found_peaks()
    
calibrate = FreqDomain_Calibration(mass_spectrum, mspeaks_results)
calibrate.ledford_calibration()
#calibrate.step_fit()
mass_spectrum.clear_molecular_formulas()


### 6. Molecular formula search

In [ ]:
from corems.molecular_id.calc.ClusterFilter import ClusteringFilter
from corems.molecular_id.search.molecularFormulaSearch import SearchMolecularFormulas

#ClusteringFilter().filter_kendrick(mass_spectrum)

mass_spectrum.molecular_search_settings.min_ppm_error  = -1
mass_spectrum.molecular_search_settings.max_ppm_error = 1
mass_spectrum.molecular_search_settings.mz_error_range = 1
mass_spectrum.molecular_search_settings.mz_error_average = 0
mass_spectrum.molecular_search_settings.min_abun_error = -30 # percentage 
mass_spectrum.molecular_search_settings.max_abun_error = 70 # percentage 

mass_spectrum.molecular_search_settings.isProtonated = True 
mass_spectrum.molecular_search_settings.isRadical= True 
mass_spectrum.molecular_search_settings.isadduct = True 

mass_spectrum.molecular_search_settings.usedAtoms['C'] = (1,90)
mass_spectrum.molecular_search_settings.usedAtoms['H'] = (4,200)
mass_spectrum.molecular_search_settings.usedAtoms['O'] = (1,20)
mass_spectrum.molecular_search_settings.usedAtoms['N'] = (0,0)
mass_spectrum.molecular_search_settings.usedAtoms['S'] = (0,0)

SearchMolecularFormulas(first_hit=True).run_worker_mass_spectrum(mass_spectrum)
print('ok')


### 7. Accessing the molecular formula  objects

In [ ]:
for mspeaks in mass_spectrum.sort_by_abundance():
   if mspeak.is_assigned: #or just if mspeak:
        for mf in mspeaks:
            print(mf.mz_calc,mf.dbe, mf.class_label, mf.string_formated)
   

In [ ]:
print (mass_spectrum.analyzer)

### 8. Export the results

In [ ]:
from corems.mass_spectrum.output.export import HighResMassSpecExport

out_filename = 'neg_esi_srfa_1ppm_test'

exportMS= HighResMassSpecExport(out_filename, mass_spectrum)
exportMS.run()

exportMS.output_type = 'csv'
exportMS.run()

exportMS.output_type = 'pandas'
exportMS.run()

# carefull if the file exits it will write another dataset to the scan number group 0
exportMS.output_type = 'hdf5'
exportMS.run()

df = exportMS.get_pandas_df()

print( df.head())